In [11]:
import os
from getpass import getpass

# Mount Google Drive
if not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')
else:
  print("Google Drive already mounted")

# Git configuration
!git config --global user.name "sammig6i"
!git config --global user.email "swghazzawi@gmail.com"

# Securely input your GitHub personal access token
token = getpass('Enter your GitHub token: ')

# Clone the GitHub repository
!git clone https://{token}@github.com/sammig6i/NBA-Fantasy-Points-Prediction.git

# Copy the notebook to the repository
!cp /content/drive/MyDrive/NBA_Fantasy_Clustering.ipynb /content/NBA-Fantasy-Points-Prediction/notebooks/

# Navigate to the repository directory
%cd /content/NBA-Fantasy-Points-Prediction

# Add the notebook file to the repository
!git add notebooks/NBA_Fantasy_Clustering.ipynb

# Commit the changes
!git commit -m "Add clustering notebook for NBA Fantasy project"

# Push the changes to GitHub securely
!git push https://{token}@github.com/sammig6i/NBA-Fantasy-Points-Prediction.git


Google Drive already mounted
Enter your GitHub token: ··········
fatal: destination path 'NBA-Fantasy-Points-Prediction' already exists and is not an empty directory.
/content/NBA-Fantasy-Points-Prediction
[main 2a8db39] Add clustering notebook for NBA Fantasy project
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite notebooks/NBA_Fantasy_Clustering.ipynb (90%)
Enumerating objects: 11, done.
Counting objects: 100% (11/11), done.
Delta compression using up to 2 threads
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 1.56 KiB | 1.56 MiB/s, done.
Total 8 (delta 6), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (6/6), completed with 3 local objects.
To https://github.com/sammig6i/NBA-Fantasy-Points-Prediction.git
   cb9dc2d..2a8db39  main -> main
